# Decomp community

In [1]:
# qiime imports
import qiime2
from qiime2 import Artifact, Metadata

# General Tool Imports
import numpy as np
import pandas as pd
import collections
from pickle import load, dump
from IPython.display import display
import warnings

# Plotting Imports
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
from statannotations.Annotator import Annotator

import itertools
import scipy
import skbio

from skbio.stats import subsample_counts
from skbio import OrdinationResults
from statsmodels.sandbox.stats.multicomp import multipletests

%matplotlib inline

In [24]:
samples = Metadata.load('13301_20200901-193839.txt').to_dataframe()

In [25]:
reads = pd.read_csv('reads_per_sample_fly_2022.csv', index_col=0)

In [26]:
# get samples with >1000 reads
samples = samples[samples.index.isin(reads[reads['0']>1000].index)]

In [27]:
samples['fly_family'].value_counts(dropna=False).to_frame()

,fly_family
Calliphoridae,747
Muscidae,23
Sarcophagidae,5


In [28]:
data = Artifact.load('table_13301.qza').view(pd.DataFrame)

In [29]:
# only get data for samples > 1000 reads
data = data[data.index.isin(samples.index)]

In [30]:
# make ra table
ra = data.apply(lambda x: x / x.sum(), axis=1)

### Import Decomposers

In [31]:
pmi_decom = pd.read_csv('../ASVs_repseq.txt',sep ='\t',index_col=3)

In [32]:
pmi_decom.head()

,#OTU ID,taxonomy,SUM,100_bp,kingdom,phylum,class,order,family,genus,species
asv,,,,,,,,,,,
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,9acc238746a1f2aa7745a0b5720c4eac,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,694120,TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTG...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,6e8986f8088b452f964e7968bb8bca87,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,372201,TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTG...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTAATTAAGTCGGATGTGAAATCCCCGAGCTCAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,9e33b8985d44f2ed1b88ac7079f70793,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,143184,TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGTA...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,d5902b1353d3aa0effaaa81cba1d1516,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,108334,TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTA...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCGCGTAGGCGGCTAATTAAGTCAAATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTGTGGGAGAGGATGGTAGAATTCCAGG,1ba09da532a0f9985b3caf72a47b42e3,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammapr...,101710,TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCG...,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,NaN


In [33]:
pmi_decom.shape

(35, 11)

In [34]:
# pmi decomposers found in fly data
fly_decomp = list(set(pmi_decom.index.tolist()).intersection(set(data.columns.tolist())))

In [35]:
len(fly_decomp)

33

In [36]:
# group samples
ra_fly = ra[fly_decomp].groupby(samples['fly_family']).mean()

In [37]:
ra_fly.T.groupby(pmi_decom['genus']).sum()

fly_family,Calliphoridae,Muscidae,Sarcophagidae
genus,,,
Acinetobacter,0.008093,0.006940,0.002817
Bacteroides,0.000373,0.000212,0.000000
Ignatzschineria,0.029716,0.040402,0.004976
Oblitimonas,0.006623,0.007615,0.000295
Peptoniphilus,0.001182,0.001621,0.000000
Savagea,0.005852,0.006447,0.000000
Vagococcus,0.078471,0.015688,0.036716
Wohlfahrtiimonas,0.005924,0.002881,0.000000


### how many samples are these decomposer asvs found in?

In [38]:
# of samples each asv is found in
pd.concat([pmi_decom.loc[pmi_decom.index.isin(fly_decomp)][['genus','species']],
           data[fly_decomp].groupby(samples['fly_family']).apply(lambda x: (x>0).sum()).T],axis=1)

,genus,species,Calliphoridae,Muscidae,Sarcophagidae
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,177,2,2
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,96,3,0
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTAATTAAGTCGGATGTGAAATCCCCGAGCTCAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,70,8,0
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,60,1,0
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCGCGTAGGCGGCTAATTAAGTCAAATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTGTGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,136,3,0
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTCTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,253,7,0
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTTTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,100,3,0
TACGAAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCGCGTAGGTGGTTTGTTAAGTTGGAAGTGAAAGCCCCGGGCTCAACCTGGGAATTGCTTTCAAAACTAGCAGGCTAGAGTACAGTAGAGGGTAGTGGAATTTCCTG,Oblitimonas,Oblitimonas alkaliphila,207,9,1
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGGAGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,200,8,3
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGAGGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,556,9,1


In [39]:
## percent of samples each ASV is found in
pd.concat([pmi_decom.loc[pmi_decom.index.isin(fly_decomp)][['genus','species']], (data[fly_decomp].groupby(samples['fly_family']).apply(
    lambda x: (x>0).sum()).T)/(data[fly_decomp].groupby(samples['fly_family']).count().T)*100], axis=1)

,genus,species,Calliphoridae,Muscidae,Sarcophagidae
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,23.694779,8.695652,40.0
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTGCGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,12.851406,13.043478,0.0
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTAATTAAGTCGGATGTGAAATCCCCGAGCTCAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,9.370817,34.782609,0.0
TACAGAGGGTGCGAGCGTTAATCGGATTTACTGGGCGTAAAGCGTACGTAGGCGGCTTTTTAAGTCGGATGTGAAATCCCTGAGCTTAACTTAGGAATTGCATTCGATACTGGGAAGCTAGAGTATGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,8.032129,4.347826,0.0
TACAGAGGGTGCAAGCGTTAATCGGATTTACTGGGCGTAAAGCGCGCGTAGGCGGCTAATTAAGTCAAATGTGAAATCCCCGAGCTTAACTTGGGAATTGCATTCGATACTGGTTAGCTAGAGTGTGGGAGAGGATGGTAGAATTCCAGG,Acinetobacter,NaN,18.206158,13.043478,0.0
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTCTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,33.868809,30.434783,0.0
TACGTAGGTGGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGCGCACGCAGGCGGCCTTTTAAGTCTGATGTGAAATCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGAAGGCTTGAGGATAGAAGAGGAAAGTGGAATTCCACG,Savagea,uncultured bacterium,13.386881,13.043478,0.0
TACGAAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCGCGTAGGTGGTTTGTTAAGTTGGAAGTGAAAGCCCCGGGCTCAACCTGGGAATTGCTTTCAAAACTAGCAGGCTAGAGTACAGTAGAGGGTAGTGGAATTTCCTG,Oblitimonas,Oblitimonas alkaliphila,27.710843,39.130435,20.0
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGGAGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,26.773762,34.782609,60.0
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTCTTTTAAGTCTGATGTGAAAGCCCTCGGCTCAACCGAGGAAGGTCATTGGAAACTGGAGGACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATG,Vagococcus,NaN,74.431058,39.130435,20.0
